In [ ]:
#!pip install tensorflow==1.13.1
#!pip install keras 2.2.4

In [24]:
import sys

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
#import pymatgen

from matplotlib import pyplot as plt

import tensorflow as tf
import keras
#from keras import backend as K
from tensorflow.keras import backend as K
from keras import initializers
from keras.layers import Dense, Input, Activation, Multiply
from keras.models import Sequential, Model, load_model
from keras.layers.merge import add, concatenate

#import deepchem as dc


In [25]:
act_dict = {0: 'linear', 1: 'multiply', 2: 'sqrt', 3: '4rt'}
np.random.seed(100000)
#np.random.seed(200000)
weight_dict = {0: 0, 1: 1, 2: np.random.uniform(0.0,1.0,1)[0], 3: 0}
bias_dict = {0: 0, 1: 1, 2: np.random.uniform(0.0,0.001,1)[0]}
nact_terms = 5
nweight_terms = 20
nbias_terms = 5

print(bias_dict)

{0: 0, 1: 1, 2: 0.0005082481908477833}


In [26]:
df = pd.read_csv('CHNO_data.csv')
#df = pd.read_csv('ExpData_v2.csv')
#df = pd.read_excel('ExpData.xlsx')
print (df.shape)

(149, 21)


In [27]:
df = df.round({'N_arb': 6, 'M_arb': 3, 'Q_arb': 3, 'Density [g/cc]': 3, 'D (exp) [km/s]': 3})
#df['D_KJ [km/s]'] = np.around(1.01*np.sqrt(df['N_arb']*np.sqrt(df['M_arb'])*np.sqrt(df['Q_arb']))*(1+1.3*df['Density [g/cc]']),decimals=3)
df

,Explosive,C,H,N,O,Al,B,Ba,Ca,Cl,...,P,Pb,Si,Heat of Formation [kcal/mol],Density [g/cc],D (exp) [km/s],N_arb,M_arb,Q_arb,D_KJ
0,"1,3,5-Triamino-2,4,6-trinitrobenzene",6.000,6.000,6.000,6.000,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-33.398140,1.800,7.350,0.029070,27.200,1089.154,7.545764
1,"1,3,5-Trinitrobenzene",6.000,3.000,3.000,6.000,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-10.399280,1.710,7.300,0.024648,32.000,1351.177,7.369484
2,"2,4,6,2',4',6'-Hexanitrodiphenylamine",12.000,5.000,7.000,12.000,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,9.882000,1.600,7.200,0.024487,32.744,1368.752,7.082862
3,"2,4,6-Trinitrocresol",7.000,5.000,3.000,7.000,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-60.288800,1.620,6.850,0.025720,29.760,1216.919,6.940381
4,"3-Nitro-1,2,4-triazole-5-one",2.000,2.000,4.000,3.000,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-24.086714,1.800,7.890,0.030769,29.500,982.410,7.720667
5,ABH,24.000,6.000,14.000,24.000,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,116.000000,1.640,7.200,0.023455,34.732,1460.412,7.270507
6,AMATEX-20 [21/41/38] [AN/RDX/TNT],3.890,5.200,4.020,5.370,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-18.030000,1.660,7.550,0.030889,27.359,1352.226,7.774580
7,AMATEX-40 [42/20/38] [AN/RDX/TNT],3.260,4.780,3.180,4.740,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-39.470000,1.610,7.010,0.031383,27.011,1273.116,7.536167
8,AMATOL-60/40 [AN/TNT],2.800,4.400,2.400,4.200,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-58.440000,1.600,5.760,0.031700,26.636,1172.334,7.362669
9,AMATOL-80/20 [AN/TNT],1.400,4.200,2.200,3.600,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,-72.920000,1.600,5.200,0.036106,25.722,1087.386,7.644200


In [28]:
inputs = np.array(df[['N_arb','M_arb','Q_arb','Density [g/cc]']])

outputs = np.array(df['D (exp) [km/s]'])
len(outputs)
Train_inputs, Test_inputs, Train_outputs, Test_outputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [29]:

def sqrt_act(x):
    return np.abs(x)**(1/2)

def frt_act(x):
    return np.abs(x)**(1/4)



In [30]:
class CustomDense(keras.layers.Layer):
    def __init__(self, num_units, input_num, activation, name, trainable_weight, trainable_bias):
        super(CustomDense, self).__init__()
        self.num_units = num_units
        self.activation = Activation(activation)
        self.trainable_weight = trainable_weight
        self.trainable_bias = trainable_bias
        self.name = name
        name_w = 'w'+self.name[1:]
        name_b = 'b'+self.name[1:]
        self.weight = self.add_weight(shape=(input_num, self.num_units), name=name_w, trainable=self.trainable_weight, initializer="zeros")
        self.bias = self.add_weight(shape=(self.num_units,), name=name_b, trainable=self.trainable_bias, initializer="zeros")
        
    def call(self, input):
        y = tf.matmul(input, self.weight) + self.bias
        y = self.activation(y)
        return y


In [63]:
def create_node(input1, input2, input3, input4, name, trainable1, trainable2, trainable3, trainable4, act, bias):
    base = name
    n1 = base + "1"
    n2 = base + "2"
    n3 = base + "3"
    n4 = base + "4"
    an1 = CustomDense(1, 1, activation = 'linear', name=n1, trainable_weight=trainable1, trainable_bias = False) (input1)
    an2 = CustomDense(1, 1, activation = 'linear', name=n2, trainable_weight=trainable2, trainable_bias = False) (input2)
    an3 = CustomDense(1, 1, activation = 'linear', name=n3, trainable_weight=trainable3, trainable_bias = False) (input3)
    an4 = CustomDense(1, 1, activation = 'linear', name=n4, trainable_weight=trainable4, trainable_bias=bias) (input4)
    if (act != "multiply"):
        an = add([an1, an2, an3, an4])
        if (act == "4rt"):
            an = Activation(frt_act) (an)
        elif (act == "sqrt"):
            an = Activation(sqrt_act) (an)
        else:
            an = Activation(act) (an)
        
    
    else:
        an = Multiply()([an1, an2, an3, an4])
        
    return an

In [64]:
def create_model(x):
    #initializer = keras.initializers.RandomUniform(minval=-0.001, maxval=0.001, seed=0)
    bias_initial = keras.initializers.Zeros()

    trainable_list = []
    for i in range(nweight_terms):
        if (x[i+nact_terms] == 2):
            trainable_list.append(True)
        else:
            trainable_list.append(False)
    
    for i in range(nbias_terms):
        if (x[i+nact_terms+nweight_terms] == 2):
            trainable_list.append(True)
        else:
            trainable_list.append(False)

    input1 = Input(shape=(1,))
    input2 = Input(shape=(1,))
    input3 = Input(shape=(1,))
    input4 = Input(shape=(1,))

    a1 = create_node(input1, input2, input3, input4, "a1", trainable_list[0], trainable_list[1], trainable_list[2], trainable_list[3], act_dict[x[0]], trainable_list[20])
    a2 = create_node(input1, input2, input3, input4, "a2", trainable_list[4], trainable_list[5], trainable_list[6], trainable_list[7], act_dict[x[1]], trainable_list[21])
    a3 = create_node(input1, input2, input3, input4, "a3", trainable_list[8], trainable_list[9], trainable_list[10], trainable_list[11], act_dict[x[2]], trainable_list[22])
    a4 = create_node(input1, input2, input3, input4, "a4", trainable_list[12], trainable_list[13], trainable_list[14], trainable_list[15], act_dict[x[3]], trainable_list[23])

    output = create_node(a1, a2, a3, a4, "output", trainable_list[16], trainable_list[17], trainable_list[18], trainable_list[19], act_dict[x[4]], trainable_list[24])
   

    model = Model(inputs=[input1, input2, input3, input4], outputs=output)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
    model.compile(loss='mse', optimizer=optimizer)
    
    layer_list = []
    for i in range(len(model.layers)):
        name = model.layers[i].name
        if ( ("activation" in name) or ("input" in name) or ("add" in name) or ("multiply" in name) ):
            continue
        else:
            layer_list.append(i)
    
    for i in range(len(layer_list)):
                
        #if (model.layers[layer_list[i]].get_weights()[0].shape==(1,1)):
         #   model.layers[layer_list[i]].set_weights( [ np.array( [[ weight_dict[x[nact_terms+i]] ]] ),  np.array( [ bias_dict[x[nact_terms+nweight_terms+i]] ] ) ] )
        #else:
         #   model.layers[layer_list[i]].set_weights( [ np.array( [ bias_dict[x[nact_terms+nweight_terms+i]] ] ),  np.array( [[ weight_dict[x[nact_terms+i]] ]] ) ] )
        
        name = model.layers[layer_list[i]].name
        if (("a14" in name) or ("a24" in name) or ("a34" in name) or ("a44" in name) or ("output4" in name)):
        
            if (model.layers[layer_list[i]].get_weights()[0].shape==(1,1)):
                model.layers[layer_list[i]].set_weights( [ np.array( [[ weight_dict[x[nact_terms+i]] ]] ),  np.array( [ bias_dict[x[nact_terms+nweight_terms+int((i+1)/5)]] ] ) ] )
            else:
                model.layers[layer_list[i]].set_weights( [ np.array( [ bias_dict[x[nact_terms+nweight_terms+int((i+1)/5)]] ] ),  np.array( [[ weight_dict[x[nact_terms+i]] ]] ) ] )
            
        else:
            model.layers[layer_list[i]].set_weights( [ np.array( [[ weight_dict[x[nact_terms+i]] ]] ), np.array( [0.] ) ] )

    #model.summary()

    return model, trainable_list

In [65]:
losses = []
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + ' Loss: ' + str(logs.get('loss')) + '                     \n')
        losses.append(logs.get('loss'))
        
class ValidLossNaN(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if np.isnan(logs.get('loss')):
            self.model.stop_training=True


def train(model, train_inputs, train_outputs, verbose=False):
    mae_es= keras.callbacks.EarlyStopping(monitor='val_loss', patience=500,
                                          min_delta=1e-4, verbose=1, mode='auto', restore_best_weights=True)
    
    terminate = keras.callbacks.TerminateOnNaN()


    EPOCHS = 10000 # Number of EPOCHS
    history = model.fit([train_inputs[:,0], train_inputs[:,1], train_inputs[:,2], train_inputs[:,3]], train_outputs, epochs=EPOCHS,
                        shuffle=False, batch_size=len(train_inputs), verbose = True, callbacks=[terminate, mae_es, ValidLossNaN()],
                        validation_split=0.2)

    if verbose:
        plt.figure()
        plt.xlabel('Epoch')
        plt.ylabel('Mean Sq Error')
        plt.plot(history.epoch, np.array(history.history['loss']),label='Training loss')
        plt.legend()
        plt.show()
    return history

In [66]:
def cv_error(individual, inputs, outputs):
    kf = KFold(n_splits=10, shuffle=True, random_state = 42)
    kf.get_n_splits(inputs)

    
    
    
    cv_mse_list = []
    train_preds_list = []
    test_preds_list = []
    train_outputs_list = []
    test_outputs_list = []
    

    for train_index, test_index in kf.split(inputs):
        #print(train_index, test_index)
        new_model, trainable = create_model(individual)
        
        train_inputs, test_inputs = inputs[train_index], inputs[test_index]
        train_outputs, test_outputs = outputs[train_index], outputs[test_index]
        
        if (any(trainable) == True):
    
            train(new_model, train_inputs, train_outputs, verbose=False)

            wt_bs = new_model.get_weights()
            weight_list = []
            bias_list = []

            for weight in wt_bs:
                if (weight.shape == (1,1)):
                    weight_list.append(weight[0])
                else:
                    bias_list.append(weight[0])
                

                #handle nan weights
            if (np.isnan(weight_list).any()):
                cv_mse = 1e50
            elif (np.isnan(np.array(bias_list)).any()):
                cv_mse = 1e50
            else:
                cv_mse = new_model.evaluate([test_inputs[:,0],test_inputs[:,1],test_inputs[:,2],test_inputs[:,3]], test_outputs)
                test_preds = np.around(new_model.predict([test_inputs[:, 0], test_inputs[:, 1], test_inputs[:, 2], test_inputs[:, 3]]),decimals=3)
                train_preds = np.around(new_model.predict([train_inputs[:, 0], train_inputs[:, 1], train_inputs[:, 2], train_inputs[:, 3]]),decimals=3)
                

                if (np.isnan(cv_mse)):
                    cv_mse = 1e50
                else:
                    cv_mse = np.around(cv_mse,decimals=6)

            cv_mse_list.append(cv_mse)
            train_preds_list.append(train_preds)
            test_preds_list.append(test_preds)
            train_outputs_list.append(train_outputs)
            test_outputs_list.append(test_outputs)
        
        else:
            wt_bs = new_model.get_weights()
            weight_list = []
            bias_list = []

            for weight in wt_bs:
                if (weight.shape == (1,1)):
                    weight_list.append(weight[0])
                else:
                    bias_list.append(weight[0])

            if (np.isnan(np.array(weight_list)).any()):
                cv_mse=1e50
            elif (np.isnan(np.array(bias_list)).any()):
                cv_mse = 1e50

            else:
                cv_mse = new_model.evaluate([test_inputs[:,0],test_inputs[:,1],test_inputs[:,2],test_inputs[:,3]], test_outputs)

                if (np.isnan(cv_mse)):
                    cv_mse = 1e50
                else:
                    cv_mse = np.around(cv_mse,decimals=6)

            cv_mse_list.append(cv_mse)
        
        
        
    print(cv_mse_list)
    return np.mean(cv_mse_list), train_preds_list, test_preds_list, train_outputs_list, test_outputs_list

In [67]:
def f3(w):
    return w

In [ ]:
#individual = [2, 3, 3, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 1, 0]  # kj
individual = [2, 3, 3, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 0, 2, 0]

### KFOLDS : 152###

#individual = [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0] # p.1

#individual = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1] # p.05

individual = [0, 0, 1, 3, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0] # p.01

#individual = [2, 0, 1, 3, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 1] # p.005

#individual = [0, 0, 3, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 1, 2, 2, 1, 2, 1, 0, 1, 0, 1] # p.001

#individual = [2, 0, 3, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 0, 0, 2, 2, 1, 1, 2, 1, 0, 0, 0, 1] # p.0005

#individual = [2, 0, 3, 0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 2, 2, 1, 1, 2, 1, 0, 0, 2, 1]  # p.0001


### KFOLDS : 190 ###

#individual = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]  # p.5

#individual = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0]  # p.1
#individual = [0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, 2] # p.01

In [ ]:
train_full_mse = new_model.evaluate([inputs[:,0],inputs[:,1],inputs[:,2],inputs[:,3]], outputs)
unseen_mse = new_model.evaluate([unseen_inputs[:,0],unseen_inputs[:,1],unseen_inputs[:,2],unseen_inputs[:,3]], unseen_outputs)

In [ ]:
mse_term, train_preds_list, test_preds_list, train_outputs_list, test_outputs_list = cv_error(individual, inputs, outputs)

In [ ]:
acts = individual[:nact_terms]
actfunc_term = 0
for i in range(nact_terms):
    actfunc_term += f3(acts[i])
    
wtbs = individual[nact_terms:]
wtbs_term = 0
for j in range(nweight_terms+nbias_terms):
    wtbs_term += f3(wtbs[j])**2
    
cplx_term = actfunc_term+wtbs_term

obj = mse_term + 0.0001*cplx_term 
print ("Individual: ", individual, flush=True)
print ("Objective function: ", mse_term, actfunc_term, wtbs_term, obj, flush=True)

In [ ]:
for i in range(len(test_outputs_list)):
   # plt.plot(train_outputs_list[i], train_preds_list[i], 'o')
    plt.plot(test_outputs_list[i], test_preds_list[i], 'o', markersize=5)

#plt.plot(df['D (exp) [km/s]'], df['D_KJ [km/s]'], 'o')

plt.xlabel('Experimental [km/s]', fontsize=20)
plt.ylabel('Predicted [km/s]', fontsize=20)
plt.title('Parity Plot', fontsize=22)

x = np.linspace(1, 10, 500)
plt.plot(x, x)
plt.ylim(1.0,10)
plt.xlim(1.0,10)

plt.rc('xtick', labelsize=18)    # fontsize of the tick labels
plt.rc('ytick', labelsize=18)    # fontsize of the tick labels


plt.grid()

In [ ]:
len(test_outputs_list)

In [ ]:
df['D_diff'] = abs(df['D (exp) [km/s]']-df['D_KJ'])
worst = df[['Explosive','D_diff','D (exp) [km/s]','D_KJ']].sort_values(by='D_diff')[-4:].reset_index(drop=True)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)

x = np.linspace(min(df['D (exp) [km/s]']), max(df['D (exp) [km/s]']), 500)
plt.plot(df['D (exp) [km/s]'], df['D_KJ'], 'o', markersize=12.0)
plt.plot(x, x)

for i in range(len(worst)):                                       # <--
    ax.annotate(str(worst['Explosive'][i]), xy=(worst['D (exp) [km/s]'][i], worst['D_KJ'][i]), fontsize=16) # <--

ax.set_xlabel('Experimental [km/s]', fontsize=24)
ax.set_ylabel('Predicted [km/s]', fontsize=24)
ax.set_title('Parity Plot', fontsize=24)

plt.rc('xtick', labelsize=20)    # fontsize of the tick labels
plt.rc('ytick', labelsize=20)    # fontsize of the tick labels

plt.ylim(1,10)
plt.xlim(1,10)
plt.grid()

In [ ]:
df['D (exp) [km/s]'].mean()

In [ ]:
rms = np.sqrt(mean_squared_error(df['D (exp) [km/s]'], df['D_KJ']))

In [ ]:
rms